<a href="https://colab.research.google.com/github/glukonatic/mystudy/blob/master/VistGroupJira_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Доустановка модуля jira
!pip install jira

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 112kB 8.1MB/s 
     |████████████████████████████████| 61kB 4.4MB/s 
     |████████████████████████████████| 2.6MB 8.2MB/s 


In [2]:
#@title Импорт библиотек

from jira import JIRA
from datetime import timedelta, datetime
import dateutil.parser
import pytz
from google.colab import files

In [3]:
#@title Введите логин и пароль к Jira

jira_login = 'anton.nagovitsin@zyfra.com'  #@param {type: "string"}
jira_pass = 'M1x@lbl4v15t'  #@param {type: "string"}


In [4]:
#@title Введите количество дней
#@markdown За сколько дней выгрузить комментарии Джиры?

days = 1  #@param {type: "number", min: 1, max: 150}

delta = timedelta(days=days)
utc=pytz.UTC

In [4]:
#@title Подключение к Jira, получение списка всех тикетов

jira_options = {'server': 'https://jira.zyfra.com'}
jira = JIRA(options=jira_options, basic_auth=(jira_login, jira_pass))
#jql = '(summary ~ "1????" OR summary ~ "2????") AND status != Done AND status != Declined'
##jql = '(summary ~ "1????" OR summary ~ "2????")'
#issues_list = jira.search_issues(jql,maxResults=1000) 

In [1]:
!pip install jira

#from jira.client import JIRA
from jira import JIRA

# Переменные для подключения к жире по API
jira_login = 'anton.nagovitsin@zyfra.com'
jira_pass = 'M1x@lbl4v15t'
jira_options = {'server': 'https://jira.zyfra.com'}

# Подключение к жире
jira = JIRA(options=jira_options, basic_auth=(jira_login, jira_pass))

# Формулируем запрос и отправляем в жиру
jql = '(summary ~ "1????" OR summary ~ "2????") AND status != Done AND status != Declined'
issues_list = jira.search_issues(jql,maxResults=1000)

print('Всего открытых задач в жире: ',len(issues_list))

# Словарь сотрудников ТП на 1 линии
visthelp2_watchers = {
    'Aleksey Polyakov' : 'aleksey.polyakov@zyfra.com',
    'Yuriy.Likhanov' : 'yuriy.likhanov@zyfra.com',
    'Anton Mulenkov' : 'anton.mulenkov@zyfra.com',
    'Pavel Khromov' : 'pavel.khromov@zyfra.com',
    'Anton Nagovitsin' : 'anton.nagovitsin@zyfra.com',
     }

# Идем по списку открытых задач в жире и ищем отсутствующих наблюдателей, исправляем
for issue in issues_list:
  if 'VISTHELP2-' in str(issue):
    break
    wemaillist = [watcher.emailAddress for watcher in jira.watchers(issue).watchers]
    for wemail in visthelp2_watchers.values():
      if wemail not in wemaillist:
        jira.add_watcher(issue,wemail)
        print("В задачу ", issue, " добавлен наблюдатель ", wemail)

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 61kB 5.4MB/s 
     |████████████████████████████████| 112kB 20.7MB/s 
     |████████████████████████████████| 2.6MB 36.0MB/s 
Всего открытых задач в жире:  199


In [6]:
#@title [to_html] Получить последний комментарий по всем тикетам, не старше введённого количества дней

filename = 'jiraReport' + datetime.now().ctime() + '.html'

filename = filename.replace(' ', '_')
filename = filename.replace(':', '')

f = open(filename, 'w')

stringBefore = []

stringBefore.append("<!DOCTYPE html>")
stringBefore.append("<html lang=\"ru\">")

stringBefore.append("<head>")
stringBefore.append("    <title>Выгрузка последних комментариев из Джиры</title>")
stringBefore.append("    <meta charset=\"utf-8\">")
stringBefore.append("    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">")
stringBefore.append("	   <style>")
stringBefore.append("   h2 { text-align: center; font-family: Verdana, Arial, Helvetica, sans-serif; color: #336; }")
stringBefore.append("   h3 { text-align: center; font-family: Verdana, Arial, Helvetica, sans-serif; color: blue; }")
stringBefore.append("   p { color: green; }")
stringBefore.append("	</style>")
stringBefore.append("</head> ")

stringBefore.append("<body>")
stringBefore.append("			<h2>Последние комментарии</h2>")
stringBefore.append('\n')
stringBefore.append("    <section id=\"docs\" style=\"margin: 5%;\">")

stringAfter = []

stringAfter.append("    </section>")
stringAfter.append("</body>")
stringAfter.append("</html> ")



for line in stringBefore:
  f.write(line)
  f.write('\n')

for i in issues_list:
  issue = jira.issue(str(i))
  if 'VISTHELP2' in str(issue):
    if len(issue.fields.comment.comments) != 0:
      lastcomment = issue.fields.comment.comments[-1]
      if dateutil.parser.isoparse(lastcomment.created) > utc.localize(datetime.today() - delta):
        f.write("		<div>")
        f.write('\n')
        f.write("			<div>")
        f.write('\n')
        f.write("				<h3><a href=\"" + "https://jira.zyfra.com/browse/" + str(issue) + "\" target=\"new\">" + issue.fields.summary + "</a></h3>")
        f.write('\n')
        f.write("				<p>" + dateutil.parser.isoparse(lastcomment.created).ctime() + "</p>")
        f.write('\n')
        f.write("				<p>" + lastcomment.author.displayName + "</p>")
        f.write('\n')
        f.write("				<p>" + lastcomment.body + "</p>")
        f.write('\n')
        f.write('='*100)
        f.write('\n')

for line in stringAfter:
  f.write(line)
  f.write('\n')

f.close()
print(filename)
files.download(filename)

jiraReportThu_Oct_15_162143_2020.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
iss = jira.issue('VISTHELP-1361')
for c in iss.fields.components:
  print(c)

In [ ]:
#iss.fields.customfield_10410 # <JIRA CustomFieldOption: value='Средняя', id='10206'>
#iss.fields.customfield_10411 # <JIRA CustomFieldOption: value='Нормальное', id='10209'>
#iss.fields.customfield_10416 # <JIRA CustomFieldOption: value='Средняя', id='10222'>
#iss.fields.customfield_10600 # <JIRA CustomFieldOption: value='АСД ГТО Карьер', id='10603'>
#iss.fields.customfield_10804 # <JIRA CustomFieldOption: value='Нет', id='10710'>
#iss.fields.customfield_11105 # <JIRA CustomFieldOption: value='1', id='10806'>
#iss.fields.customfield_11331 # <JIRA CustomFieldOption: value='No', id='10969'>
#iss.fields.customfield_11332 # <JIRA CustomFieldOption: value='No', id='10971'>
#iss.fields.assignee # <JIRA User: displayName='VIST Support', key='vist.support@zyfra.com', name='vistsupport24@zyfra.com'>

In [79]:
from jira.client import JIRA

iss = jira.issue('VISTHELP-1361')
jira.add_watcher(iss,'anton.nagovitsin@zyfra.com')

In [ ]:
watchers = ['anton.nagovitsin@zyfra.com','pavel.khromov@zyfra.com','anton.mulenkov@zyfra.com','yuriy.likhanov@zyfra.com','aleksey.polyakov@zyfra.com','mikhail.serba@zyfra.com']


for i in issues_list:
  issue = jira.issue(str(i))
  if 'VISTHELP2-' in str(issue):
    if jira.watchers

In [218]:
jql2 = '(summary ~ "1????" OR summary ~ "2????") AND status != Done AND status != Declined'
#jql = '(summary ~ "1????" OR summary ~ "2????")'
issues_list2 = jira.search_issues(jql2,maxResults=1000) 

len(issues_list2)

199

In [2]:
!pip install jira

from jira.client import JIRA
#from jira import JIRA

# Переменные для подключения к жире по API
jira_login = 'anton.nagovitsin@zyfra.com'
jira_pass = 'M1x@lbl4v15t'
jira_options = {'server': 'https://jira.zyfra.com'}

# Подключение к жире
jira = JIRA(options=jira_options, basic_auth=(jira_login, jira_pass))

# Формулируем запрос и отправляем в жиру
jql = '(summary ~ "1????" OR summary ~ "2????") AND status != Done AND status != Declined'
issues_list = JIRA.search_issues?(jql_str=jql,maxResults=1000)

print('Всего открытых задач в жире: ',len(issues_list))

# Словарь сотрудников ТП на 1 линии
visthelp2_watchers = {
    'Aleksey Polyakov' : 'aleksey.polyakov@zyfra.com',
    'Yuriy.Likhanov' : 'yuriy.likhanov@zyfra.com',
    'Anton Mulenkov' : 'anton.mulenkov@zyfra.com',
    'Pavel Khromov' : 'pavel.khromov@zyfra.com',
    'Anton Nagovitsin' : 'anton.nagovitsin@zyfra.com',
     }

#      
for i in issues_list:
  
  issue = jira.issue(str(i))
  print(i, issue)
  if 'VISTHELP2-' in str(issue):
    wemaillist = [watcher.emailAddress for watcher in jira.watchers(issue).watchers]
    for wemail in visthelp2_watchers.values():
      if wemail not in wemaillist:
        jira.add_watcher(issue,wemail)
        print("В задачу ", issue, " добавлен наблюдатель ", wemail)

In [126]:
visthelp2_watchers = {
    'Aleksey Polyakov' : 'aleksey.polyakov@zyfra.com',
    'Yuriy.Likhanov' : 'yuriy.likhanov@zyfra.com',
    'Anton Mulenkov' : 'anton.mulenkov@zyfra.com',
    'Pavel Khromov' : 'pavel.khromov@zyfra.com',
    'Anton Nagovitsin' : 'anton.nagovitsin@zyfra.com',
     }

In [130]:
for k,v in visthelp2_watchers.items():
  print(k,v)

Aleksey Polyakov aleksey.polyakov@zyfra.com
Yuriy.Likhanov yuriy.likhanov@zyfra.com
Anton Mulenkov anton.mulenkov@zyfra.com
Pavel Khromov pavel.khromov@zyfra.com
Anton Nagovitsin anton.nagovitsin@zyfra.com


In [ ]:
'Mikhail Serba', 'mikhail.serba@zyfra.com', 'mikhail.serba@zyfra.com'
'Aleksey Polyakov', 'polyakov_av@vistgroup.ru', 'aleksey.polyakov@zyfra.com'
'Yuriy.Likhanov', 'yuriy.likhanov@zyfra.com', 'yuriy.likhanov@zyfra.com'
'Anton Mulenkov', 'anton.mulenkov@zyfra.com', 'anton.mulenkov@zyfra.com'
'Pavel Khromov', 'khromov@vistgroup.ru', 'pavel.khromov@zyfra.com'
'Anton Nagovitsin', 'nagovitsin@vistgroup.ru', 'anton.nagovitsin@zyfra.com'
'Sergey Kondratyev', 'kondratyev@vistgroup.ru', 'sergey.kondratyev@zyfra.com'
'VIST Support', 'vist.support@zyfra.com', 'vistsupport24@zyfra.com'

In [ ]:
for i in issues_list:
  issue = jira.issue(str(i))
  if 'VISTHELP' in str(issue):
    print(issue)
    print(issue.fields.creator)
    for field in issue.fields:
      print(type(field))
    
    break


  #if 'VISTV8' in str(issue):
    #if len(issue.fields.comment.comments) != 0:
    #  for comment in issue.fields.comment.comments:
    #    if 'idletype' in comment.body:
    #      print(issue)
    #      print(comment.body)
      #lastcomment = issue.fields.comment.comments[-1]
      #print(lastcomment.author.displayName)
      #if dateutil.parser.isoparse(lastcomment.created) > utc.localize(datetime.today() - delta):
        

VISTHELP2-2067
Anton Nagovitsin


TypeError: ignored

In [ ]:
#@title [to_html] Все комменты от пользователя

#@markdown Введите имя и фамилию пользователя, как в Джире

username = 'Dmitriy Protasov' #@param {type: "string"}

filename = 'jiraReport' + datetime.now().ctime() + '.html'

filename = filename.replace(' ', '_')
filename = filename.replace(':', '')

f = open(filename, 'w')

stringBefore = []

stringBefore.append("<!DOCTYPE html>")
stringBefore.append("<html lang=\"ru\">")

stringBefore.append("<head>")
stringBefore.append("    <title>Выгрузка комментариев от Вовы из Джиры</title>")
stringBefore.append("    <meta charset=\"utf-8\">")
stringBefore.append("    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">")
stringBefore.append("	   <style>")
stringBefore.append("   h2 { text-align: center; font-family: Verdana, Arial, Helvetica, sans-serif; color: #336; }")
stringBefore.append("   h3 { text-align: center; font-family: Verdana, Arial, Helvetica, sans-serif; color: blue; }")
stringBefore.append("   p { color: green; }")
stringBefore.append("	</style>")
stringBefore.append("</head> ")

stringBefore.append("<body>")
stringBefore.append("			<h2>Последние комментарии</h2>")
stringBefore.append('\n')
stringBefore.append("    <section id=\"docs\" style=\"margin: 5%;\">")

stringAfter = []

stringAfter.append("    </section>")
stringAfter.append("</body>")
stringAfter.append("</html> ")



for line in stringBefore:
  f.write(line)
  f.write('\n')

#hw = True

for i in issues_list:
  issue = jira.issue(str(i))
  #if 'VISTHELP2' in str(issue):
  if len(issue.fields.comment.comments) != 0:
#    hw = True
    for comment in issue.fields.comment.comments:
      if comment.author.displayName == username:
        #print(comment.body)
        f.write("		<div>")
        f.write('\n')
        f.write("			<div>")
        f.write('\n')
#        if hw:
        f.write("				<h3><a href=\"" + "https://jira.zyfra.com/browse/" + str(issue) + "\" target=\"new\">" + issue.fields.summary + "</a></h3>")
        f.write('\n')
        f.write("				<p>" + dateutil.parser.isoparse(comment.created).ctime() + "</p>")
        f.write('\n')
        f.write("				<p>" + comment.author.displayName + "</p>")
        f.write('\n')
        f.write("				<p>" + comment.body + "</p>")
        f.write('\n')
        f.write('='*100)
        f.write('\n')
#        hw = False




for line in stringAfter:
  f.write(line)
  f.write('\n')

f.close()
print(filename)
files.download(filename)

jiraReportWed_Aug_19_135935_2020.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Получить последний комментарий по всем тикетам, не старше введённого количества дней

filename = 'jiraReport' + datetime.now().ctime() + '.txt'

filename = filename.replace(' ', '_')
filename = filename.replace(':', '')

f = open(filename, 'w')

for i in issues_list:
  issue = jira.issue(str(i))
  if 'VISTHELP2' in str(issue):
    if len(issue.fields.comment.comments) != 0:
      lastcomment = issue.fields.comment.comments[-1]
      if dateutil.parser.isoparse(lastcomment.created) > utc.localize(datetime.today() - delta):
        f.write('\n')
        f.write(str(issue))
        f.write('\n')
        f.write('https://jira.zyfra.com/browse/' + str(issue))
        f.write('\n')
        f.write('-'*10)
        f.write('\n')
        f.write(issue.fields.summary)
        f.write('\n')
        f.write('-'*30)
        f.write('\n')
        f.write(dateutil.parser.isoparse(lastcomment.created).ctime())
        f.write('\n')
        f.write('-'*60)
        f.write('\n')
        f.write(lastcomment.author.displayName)
        f.write('\n')
        f.write('-'*90)
        f.write('\n')
        f.write(lastcomment.body)
        f.write('\n')
        f.write('='*100)
        f.write('\n')
        f.write('='*100)
        f.write('\n')
        f.write('\n')


f.close()
print(filename)
files.download(filename)

jiraReportThu_Jun_25_163722_2020.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Получить последний комментарий по всем тикетам, не старше введённого количества дней, не принадлежащий сотруднику ТП

filename = 'TP_jiraReport' + datetime.now().ctime() + '.txt'

filename = filename.replace(' ', '_')
filename = filename.replace(':', '')

tpNamesList = ['Anton Nagovitsin','Aleksey Polyakov','Pavel Khromov','Sergey Kondratyev','Anton Mulenkov','Yuriy.Likhanov']

f = open(filename, 'w')

for i in issues_list:
  issue = jira.issue(str(i))
  if 'VISTHELP2' in str(issue):
    if len(issue.fields.comment.comments) != 0:
      lastcomment = issue.fields.comment.comments[-1]
      if lastcomment.author.displayName not in tpNamesList:
        if dateutil.parser.isoparse(lastcomment.created) > utc.localize(datetime.today() - delta):
          f.write('\n')
          f.write(str(issue))
          f.write('\n')
          f.write('https://jira.zyfra.com/browse/' + str(issue))
          f.write('\n')
          f.write('-'*10)
          f.write('\n')
          f.write(issue.fields.summary)
          f.write('\n')
          f.write('-'*30)
          f.write('\n')
          f.write(dateutil.parser.isoparse(lastcomment.created).ctime())
          f.write('\n')
          f.write('-'*60)
          f.write('\n')
          f.write(lastcomment.author.displayName)
          f.write('\n')
          f.write('-'*90)
          f.write('\n')
          f.write(lastcomment.body)
          f.write('\n')
          f.write('='*100)
          f.write('\n')
          f.write('='*100)
          f.write('\n')
          f.write('\n')


f.close()
print(filename)
files.download(filename)

TP_jiraReportFri_Aug_21_112600_2020.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
filename = 'jiraReport' + datetime.now().ctime() + '.html'

filename = filename.replace(' ', '_')
filename = filename.replace(':', '')

f = open(filename, 'w')

stringBefore = []

stringBefore.append("<!DOCTYPE html>")
stringBefore.append("<html lang=\"ru\">")

stringBefore.append("<head>")
stringBefore.append("    <title>Выгрузка последних комментариев из Джиры</title>")
stringBefore.append("    <meta charset=\"utf-8\">")
stringBefore.append("    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">")
stringBefore.append("	   <style>")
stringBefore.append("   h2 { text-align: center; font-family: Verdana, Arial, Helvetica, sans-serif; color: #336; }")
stringBefore.append("   h3 { text-align: center; font-family: Verdana, Arial, Helvetica, sans-serif; color: blue; }")
stringBefore.append("   p { color: green; }")
stringBefore.append("	</style>")
stringBefore.append("</head> ")

stringBefore.append("<body>")
stringBefore.append("			<h2>Последние комментарии</h2>")
stringBefore.append('\n')
stringBefore.append("    <section id=\"docs\" style=\"margin: 5%;\">")

stringAfter = []

stringAfter.append("    </section>")
stringAfter.append("</body>")
stringAfter.append("</html> ")



for line in stringBefore:
  f.write(line)
  f.write('\n')


tpNamesList = ['Anton Nagovitsin','Aleksey Polyakov','Pavel Khromov','Sergey Kondratyev','Anton Mulenkov','Yuriy.Likhanov']

f = open(filename, 'w')

for i in issues_list:
  issue = jira.issue(str(i))
  if 'VISTHELP2' in str(issue):
    if len(issue.fields.comment.comments) != 0:
      lastcomment = issue.fields.comment.comments[-1]
      if lastcomment.author.displayName not in tpNamesList:
        if dateutil.parser.isoparse(lastcomment.created) > utc.localize(datetime.today() - delta):
          f.write("		<div>")
          f.write('\n')
          f.write("			<div>")
          f.write('\n')
          f.write("				<h3><a href=\"" + "https://jira.zyfra.com/browse/" + str(issue) + "\" target=\"new\">" + issue.fields.summary + "</a></h3>")
          f.write('\n')
          f.write("				<p>" + dateutil.parser.isoparse(lastcomment.created).ctime() + "</p>")
          f.write('\n')
          f.write("				<p>" + issue.fields.summary + "</p>")
          f.write('\n')
          f.write("				<p>" + lastcomment.author.displayName + "</p>")
          f.write('\n')
          f.write("				<p>" + lastcomment.body + "</p>")
          f.write('\n')
          f.write('='*100)
          f.write('\n')


for line in stringAfter:
  f.write(line)
  f.write('\n')

f.close()
print(filename)
files.download(filename)

jiraReportFri_Aug_21_113450_2020.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title [-] Получить комментарии тикетов 1
for i in issues_list:
  issue = jira.issue(str(i))
  #print('Тикет ',issue)
  allcomments = issue.fields.comment.comments
  for comment in allcomments:
    commentTimeISO = dateutil.parser.isoparse(comment.created)
    if commentTimeISO > utc.localize(datetime.today() - delta):
      commentTime = commentTimeISO.ctime()
      print(issue.fields.summary)
      print(commentTimeISO)
      print(comment.author.displayName)
      print(comment.body)

In [ ]:
for issue in issues_list:
  print(str(issue))